In [1]:
from PIL import Image
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
import pandas as pd
import os
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
create_data_folders = False

In [3]:
# path to the model weights files.
weights_path = '/media/tyler/slowdata/models/keras_vgg_16'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/home/tyler/data/kaggle/dog_breed/train/'
ordered_train_folder = "/home/tyler/data/kaggle/dog_breed/train_ordered/"
test_data_dir = '/home/tyler/data/kaggle/dog_breed/test/'
labels = '/home/tyler/data/kaggle/dog_breed/labels.csv'

epochs = 50
batch_size = 16
nb_train_samples = 10222

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False,
                          input_shape=(img_width, img_height, 3), classes=120)
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(120, activation='softmax'))

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Model loaded.


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
labels_df = pd.read_csv(labels)
class_names = labels_df['breed'].unique()
class_to_index = {c:i for i, c in enumerate(class_names)}
index_to_class = {i:c for c, i in class_to_index.items()}
n_classes = len(class_names)
print("{} classes".format(n_classes))

120 classes


In [6]:
if create_data_folders:
    created_folder_names = []
    for train_img_tuple in labels_df.values:
        file_name = train_img_tuple[0] + ".jpg"
        label_value = train_img_tuple[1]
        if label_value not in created_folder_names:
            created_folder_names.append(label_value)
            os.mkdir(ordered_train_folder + label_value)
        copyfile(train_data_dir + file_name, ordered_train_folder + label_value
                 + "/" + file_name)

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    ordered_train_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


# fine-tune the model
checkpointer = ModelCheckpoint(filepath='./tmp/weights.{epoch:02d}.hdf5', verbose=1)

model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    callbacks=[checkpointer])

Found 10222 images belonging to 120 classes.


/home/tyler/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, callbacks=[<keras.ca..., steps_per_epoch=638)`


Epoch 1/50
 49/638 [=>............................] - ETA: 78s - loss: 5.4779 - acc: 0.0102

KeyboardInterrupt: 